In [56]:
!pip install huggingface_hub==0.19.4


  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.34.2
    Uninstalling huggingface-hub-0.34.2:
      Successfully uninstalled huggingface-hub-0.34.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 1.9.0 requires huggingface_hub>=0.21.0, but you have huggingface-hub 0.19.4 which is incompatible.
langchain-huggingface 0.3.1 requires huggingface-hub>=0.33.4, but you have huggingface-hub 0.19.4 which is incompatible.
transformers 4.54.0 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 0.19.4 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Literal, Annotated
from langchain_core.messages import SystemMessage, HumanMessage
import operator
from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.output_parsers import PydanticOutputParser,StrOutputParser
import os 
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [5]:
print("Token loaded?", "HUGGINGFACEHUB_API_TOKEN" in os.environ)

Token loaded? True


In [7]:
from huggingface_hub import HfApi
import os

from dotenv import load_dotenv
load_dotenv()

token = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
api = HfApi(token=token)

# Check if you can access the model
model_info = api.model_info("google/flan-t5-base")
print(model_info)


ModelInfo(id='google/flan-t5-base', author='google', sha='7bcac572ce56db69c1ea7c8af255c5d7c9672fc2', last_modified=datetime.datetime(2023, 7, 17, 12, 48, 39, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=1597836, likes=953, library_name='transformers', tags=['transformers', 'pytorch', 'tf', 'jax', 'safetensors', 't5', 'text2text-generation', 'en', 'fr', 'ro', 'de', 'multilingual', 'dataset:svakulenk0/qrecc', 'dataset:taskmaster2', 'dataset:djaym7/wiki_dialog', 'dataset:deepmind/code_contests', 'dataset:lambada', 'dataset:gsm8k', 'dataset:aqua_rat', 'dataset:esnli', 'dataset:quasc', 'dataset:qed', 'arxiv:2210.11416', 'arxiv:1910.09700', 'license:apache-2.0', 'text-generation-inference', 'endpoints_compatible', 'region:us'], pipeline_tag=None, mask_token=None, card_data={'language': ['en', 'fr', 'ro', 'de', 'multilingual'], 'license': 'apache-2.0', 'library_name': None, 'tags': ['text2text-generation'], 'datasets': ['svakulenk0/qrecc', 'taskmaster2'

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()  # This should load the .env file

token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
print("Token loaded:", bool(token))
print("Token preview:", token[:6], "..." if token else None)


Token loaded: True
Token preview: hf_jQs ...


In [9]:
# Load modeln
generator_llm = HuggingFaceEndpoint(
    repo_id="google/flan-t5-base",             # the model you want
    task="text2text-generation",               # explicitly declare your task
    huggingfacehub_api_token=token,            # now your token is passed in
    temperature=0.7,
    max_length=100
)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_6872\3551252411.py:2: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  generator_llm = HuggingFaceEndpoint(
WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\lenovo\.cache\huggingface\token
Login successful


In [10]:
evaluator_llm = HuggingFaceEndpoint(
    repo_id="google/t5-v1_1-small",            # your chosen model
    task="text2text-generation",               # <— must specify task
    huggingfacehub_api_token=token,            # your HF token
    temperature=0.7,
    max_length=100
)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\lenovo\.cache\huggingface\token
Login successful


In [11]:

optimizer_llm = HuggingFaceEndpoint(
    repo_id="google/flan-t5-large",            # your chosen model
    task="text2text-generation",               # <— must specify task
    huggingfacehub_api_token=token,            # your HF token
    temperature=0.7,
    max_length=100
)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\lenovo\.cache\huggingface\token
Login successful


In [12]:
from pydantic import BaseModel, Field

# Step : Define structure
class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")

In [13]:
# create the parser
parser = PydanticOutputParser(pydantic_object=TweetEvaluation)


In [14]:
# Update your prompt to include parser formatting instructions
format_instructions = parser.get_format_instructions()


In [15]:
# state
class TweetState(TypedDict):

    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int

In [16]:
def generate_tweet(state: TweetState):

    # prompt
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ] 

    # send generator_llm
    response = generator_llm.invoke(messages).content

    # return response
    return {'tweet': response, 'tweet_history': [response]}

In [17]:
def evaluate_tweet(state: TweetState):

    # prompt 
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
{format_instructions}
""")
    ]
    
    # send evaluator_llm
    response = evaluator_llm.invoke(messages)
    
    # Parse the response into a structured object
    try:
        parsed = parser.parse(response)
    except Exception as e:
        print("Parsing failed:", e)
        print("Raw response:", response)
        raise e

    return {
        "evaluation": parsed.evaluation,
        "feedback": parsed.feedback,
        "feedback_history": [parsed.feedback],
    }

In [18]:
def optimize_tweet(state: TweetState):

    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1

    return {'tweet': response, 'iteration': iteration, 'tweet_history': [response]}

In [19]:
def route_evaluation(state: TweetState):

    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

In [20]:
graph = StateGraph(TweetState)

# create nodes
graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)

# create edges 
graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')

graph.add_conditional_edges('evaluate', route_evaluation, {'approved': END, 'needs_improvement': 'optimize'})
graph.add_edge('optimize', 'evaluate')

# compile
workflow = graph.compile()


In [21]:
workflow

KeyboardInterrupt: 

In [21]:
initial_state = {
    "topic": "Jobs in Tech",
    "iteration": 1,
    "max_iteration": 5
}


#Final notes:
1. Compatibility issues to be resolved. 
2.  Review of entire code (specifically structured output and invoke)